###todo: add more variables from public data : time?
###change settings on SVR

### Sky cover, relative humidity, precipitation, temperature, dew point, wind speed

Short term weather forecasts for next day as well as for next three days

 Photovoltaic, meteorological and radiative measurements at SIRTA
 ####Col 7: Measured solar irradiance on the plane of the panels (27.7�)(zone 1)
 ####Col 12: 2-m Air temperature (zone 1)
 ####Col 14: 10-m Wind speed (zone 1)

In [2]:
# IMPORT MODULES

%matplotlib inline

import datetime
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dts
import calendar

import sklearn.preprocessing as pp
from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import r2_score

import pandas as pd

In [5]:
# READ DATA
data = pd.read_csv("meteo_france_marmai2015.csv", sep=";")
data
#7149 = orly

,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,...,nnuage2,ctype2,hnuage2,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4,Unnamed: 59
0,7005,20150315000000,102690,-80,8,40,3.600000,276.850000,275.350000,90,...,mq,mq,mq,mq,mq,mq,mq,mq,mq,NaN
1,7015,20150315000000,102720,-50,8,30,4.200000,276.750000,276.050000,95,...,mq,mq,mq,mq,mq,mq,mq,mq,mq,NaN
2,7020,20150315000000,102880,-50,8,50,8.100000,280.050000,276.150000,76,...,mq,mq,mq,mq,mq,mq,mq,mq,mq,NaN
3,7027,20150315000000,102710,-70,7,60,1.500000,277.750000,275.150000,83,...,3,mq,4560,mq,mq,mq,mq,mq,mq,NaN
4,7037,20150315000000,102650,-50,8,20,3.200000,276.950000,275.450000,90,...,8,mq,1050,mq,mq,mq,mq,mq,mq,NaN
5,7072,20150315000000,102470,-20,8,40,3.100000,278.250000,274.950000,79,...,mq,mq,mq,mq,mq,mq,mq,mq,mq,NaN
6,7110,20150315000000,102810,-70,8,40,4.000000,279.850000,275.150000,72,...,mq,mq,mq,mq,mq,mq,mq,mq,mq,NaN
7,7117,20150315000000,102780,-80,8,60,7.800000,280.350000,275.250000,70,...,mq,mq,mq,mq,mq,mq,mq,mq,mq,NaN
8,7130,20150315000000,102610,-90,8,10,4.200000,278.250000,275.050000,80,...,mq,mq,mq,mq,mq,mq,mq,mq,mq,NaN
9,7139,20150315000000,102580,-50,8,40,4.500000,277.450000,275.150000,85,...,mq,mq,mq,mq,mq,mq,mq,mq,mq,NaN


In [30]:
dataset = np.array(data[["hh(UTC)", "IrrPOA(W/m2)"]]).astype(np.float)

#Remove nan and infinite values
masknan = ~np.any(np.isnan(dataset), axis=1)
dataset = dataset[masknan]
maskfin = np.any(np.isfinite(dataset), axis=1)
dataset = dataset[maskfin]
X=dataset[:,0]
y=dataset[:,1]

X = np.reshape(X,(len(X),1))

#SCALE INDEPENDENT DATA - no need here, because time is regular?
#X=pp.StandardScaler().fit_transform(X)

In [34]:
X_training, X_test, y_training, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10]}]

In [ ]:
print("# Tuning hyper-parameters for R2 Score")

clf = GridSearchCV(SVR, parameters, scoring='r2')
clf.fit(X_training, y_training)

print("Best parameters set found on development set:")
print(clf.best_params_)

print("R2 Score:")
y_true, y_pred = y_test, clf.predict(X_test)
print(r2_score(y_true, y_pred))

print("Grid Scores:")
print(clf.grid_scores)